In [7]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import math
from functools import partial
from open3d.t.geometry import TriangleMesh
import util
import time
import matplotlib.cm as cm
from tqdm import tqdm
from multiprocessing import Pool

In [8]:
dataname = "C:/Users/chris/Desktop/Documents/NewData/CircularVentilationGrateExtraCleanedFull.ply"
pcd = o3d.io.read_point_cloud(dataname)
pcd = util.preProcessCloud(pcd)
kdtree = o3d.geometry.KDTreeFlann(pcd)
points = np.asarray(pcd.points)

In [ ]:
def find_normal(i):
    """ Compute PCA for a single point and return the normal direction. """
    _, idx, _ = kdtree.search_knn_vector_3d(points[i], 30)
    neighbors = points[idx]
    centered = neighbors - np.mean(neighbors, axis=0)
    cov = np.cov(centered.T)
    eig_val, eig_vec = np.linalg.eigh(cov)
    sorted_idx = np.argsort(eig_val)[::-1]
    eig_vec = eig_vec[:, sorted_idx]
    return eig_vec[:, 2]  # Smallest eigenvector as normal

def compute_eigenvalues_parallel(pcd):
    n = len(pcd.points)
    points = np.asarray(pcd.points)
    plane_directions = np.zeros((n, 3))
    """ Compute eigenvalues for all points using parallel processing. """
    pool = Pool(processes=4)
    plane_directions = pool.map(find_normal, range(n))
    
    pool.close()
    pool.join()

    return np.array(plane_directions)

plane_directions = compute_eigenvalues_parallel(pcd)

In [ ]:
# def compute_eigenvalues_open3d(pcd, k=30):
#     """ Compute eigenvalues for all points using Open3D KD-tree. """
#     # Convert point cloud to NumPy array
#     points = np.asarray(pcd.points)
#     n = len(points)
#     # Create KD-Tree
#     kdtree = o3d.geometry.KDTreeFlann(pcd)
#     # Preallocate array for eigenvalues
#     plane_directions = []
#     for i in tqdm(range(n)):
#         #neighbor_coordinates = get_neighbors(points[i], pcd, kdtree, radius=1.2)
#         _, idx, _ = kdtree.search_knn_vector_3d(points[i], k)
#         neighbors = points[idx]
#         centered = neighbors - np.mean(neighbors, axis=0)
#         cov = np.cov(centered.T)
#         eig_val, eig_vec = np.linalg.eigh(cov)
#         sorted_idx = np.argsort(eig_val)[::-1]
#         #eig_val = eig_val[sorted_idx]
#         eig_vec = eig_vec[:, sorted_idx]
#         #eig_val_norm = eig_val.copy()

#         #for z in range(len(eig_val)):
#             #eig_val_norm[z] = np.exp(eig_val[z])/np.sum(np.exp(eig_val))

#         #plane_direction = np.cross(eig_vec[:, 0], eig_vec[:, 1])
#         plane_directions.append(eig_vec[:, 2])

#     return plane_directions

# plane_directions = compute_eigenvalues_open3d(pcd, k=30)
# plane_directions = np.array(plane_directions)
# print(f'Plane directions size: {plane_directions.shape}')

100%|██████████| 2114266/2114266 [05:07<00:00, 6879.37it/s] 


Plane directions size: (2114266, 3)
